# Multiline Queues

In [2]:
import itertools 
import copy
from sage.combinat.q_analogues import *
from sage.combinat.sf.sf import *
from sage.combinat.ncsf_qsym.qsym import *
from sage.rings.rational_field import *
from sage.combinat.subset import *
from sage.combinat.permutation import *
from sage.misc.latex import *

import sage.combinat.permutation as permutation
from sage.combinat.sf.macdonald import qt_kostka
from sage.combinat.sf.ns_macdonald import E
from sage.rings.polynomial.polydict import ETuple

coeffs_ring = ZZ['q','t']

q = coeffs_ring.gens()[0]
t = coeffs_ring.gens()[1]

coeffs_field = coeffs_ring.fraction_field()
R=PolynomialRing(coeffs_field,11,'x')
xs=list(R.gens())

sym = SymmetricFunctions(coeffs_field)
h = sym.homogeneous()
m = sym.monomial()
s = sym.schur()
e = sym.elementary()
p = sym.power()
Ht = sym.macdonald().Ht()
H = sym.macdonald().H()
MP = sym.macdonald().P()
W = sym.macdonald(t=0).P()
MJ = sym.macdonald().J()
HLP = sym.hall_littlewood(t).P()
HLQ = sym.hall_littlewood(t).Q();
HLQp = sym.hall_littlewood(t).Qp();
JJ = sym.jack().J()
qsym = QuasiSymmetricFunctions(coeffs_field)
F = qsym.Fundamental()
M = qsym.Monomial()
QS = qsym.QS()
YQS = qsym.YQS()

## Object construction

In [3]:
# COORDINATES START AT 1
# Coordinates are given in the usual way (i,j) means i to the right, j up

def balls_coordinates(balls):
    coords = []
    w = len(balls)
    l = len(balls[0])
    for i in range(w):
        for j in range(l):
            if balls[i][j] == 1: 
                coords.append([i+1,j+1])
            elif balls[i][j] == 2:
                coords.append([i+1,j+1,"b"])
            elif balls[i][j] == -2:
                coords.append([i+1,j+1,"r"]) 
    return(coords)
    
def balls_is_valid(balls):
    return(True)

# find the queue index in which elem is
def find_queue(self,elem):
    ind = -1
    for i in range(len(self.queues)):
        queue = self.queues[i]
        if elem in queue:
            ind = i
            break
    if ind == -1:
        return("element not queued")
    else:
        return(ind)  
    
# word is a ternary word with 0,1,2 and 1=(, 2=) and 0=blank, to perform parenthesis matching algorithm
# returns the leftover letters from row1 after matching all possible 1s and 2s, and deleting unmatched 2s
def match_parenthesis(w):
    word = w.copy()
    boo = True
    while boo:
        p1 = -1
        p2 = -1
        for i in range(len(word)):
            if word[i] == 1:
                p1 = i
            elif word[i] == 2:
                p2 = i
            if p2 != -1:
                word[p2] = 0
                if p1 != -1:
                    word[p1] = 0        
                break
        if p2==-1:
            boo = False
    return(word)

# word is a ternary word with 0,1,2 and 1=(, 2=) and 0=blank, to perform parenthesis matching algorithm
# returns the leftover letters from row1 after matching all possible 1s and 2s.
def match_parenthesis_cyclic(w):
    word = w.copy()
    boo = True
    while boo:
        # First do normal matchings
        p1 = -1
        p2 = -1
        rep = True
        while(rep):
            for i in range(len(word)):
                if word[i] == 1:
                    p1 = i
                elif word[i] == 2:
                    p2 = i
                if p1 != -1 and p2 != -1 and p1<p2:
                    word[p2] = 0
                    word[p1] = 0  
                    p1=-1
                    p2=-1
                    rep = True
                    break
                else:
                    rep = False
        #Check for cyclic matchings        
        p1 = -1
        p2 = -1
        for i in (range(len(word))):
            if word[i] == 1:
                p1 = i
        for i in range(len(word)):
            if word[i] == 2:
                p2 = i
                
        if p1 != -1 and p2 != -1:
            word[p2] = 0
            word[p1] = 0
            
        else:
            boo = False
    return(word)
    
# collapse row 1 on top of row 2
# this method updates row1 and row2
# wit is a boolean variable that says if a collapse occurred or not
def two_row_collapse(row1,row2):
    par_word = []
    wit = False
    # Create the parenthesis word to do the matching algorithm
    for i in range(len(row1)):
        if row1[i]==1:
            par_word.append(1)
        else:
            par_word.append(0)
        if row2[i]==1:
            par_word.append(2)
        else:
            par_word.append(0)
    # Apply parenthesis matching
    red_par_word = match_parenthesis(par_word)
    # Find indices on row1 to collapse
    coll = [red_par_word[i] for i in range(len(red_par_word)) if i%2==0]    
    for i in range(len(row1)):
        if coll[i] == 1:
            wit = True
            row1[i] = 0
            row2[i] = 1
    return(row1.copy(),row2.copy(),wit)

# collapse cyclically row 1 on top of row 2
# this method updates row1 and row2
# wit is a boolean variable that says if a collapse occurred or not
def two_row_collapse_cyclic(row1,row2):
    par_word = []
    wit = False
    # Create the parenthesis word to do the matching algorithm
    for i in range(len(row1)):
        if row1[i]==1:
            par_word.append(1)
        else:
            par_word.append(0)
        if row2[i]==1:
            par_word.append(2)
        else:
            par_word.append(0)
    # Apply parenthesis matching
    red_par_word = match_parenthesis_cyclic(par_word)
    # Find indices on row1 to collapse
    coll = [red_par_word[i] for i in range(len(red_par_word)) if i%2==0]    
    for i in range(len(row1)):
        if coll[i] == 1:
            wit = True
            row1[i] = 0
            row2[i] = 1
    return(row1.copy(),row2.copy(),wit)



# collapse row 1 on top of row 2
# this method updates row1 and row2
# wit is a boolean variable that says if a collapse occurred or not
def two_row_collapse_game(row1,row2):
    par_word = []
    wit = False
    # Create the parenthesis word to do the matching algorithm
    for i in range(len(row1)):
        if row1[i][0]==1:
            par_word.append(1)
        else:
            par_word.append(0)
        if row2[i][0]==1:
            par_word.append(2)
        else:
            par_word.append(0)
    # Apply parenthesis matching
    red_par_word = match_parenthesis(par_word)
    # Find indices on row1 to collapse
    coll = [red_par_word[i] for i in range(len(red_par_word)) if i%2==0]    
    for i in range(len(row1)):
        if coll[i] == 1:
            wit = True
            element_row1 = row1[i]
            
            row1[i] = [0,""]
            row2[i] = element_row1
    return(row1.copy(),row2.copy(),wit)

###  SHIFTED CASE METHODS  ###
        
# Shifted parenthesis matching
   
# word is a ternary word with 0,1,2 and 1=(, 2=) and 0=blank, to perform parenthesis matching algorithm
# returns the leftover letters from row1 after matching all possible 1s and 2s, and deleting unmatched 2s
def shifted_match_parenthesis(w,first):
    word = w.copy()
    pos = -1
    boo = True
    while boo:
        p1 = -1
        p2 = -1
        for i in range(len(word)):
            if word[i][0] == 1:
                p1 = i
            if word[i][0] == 2:
                p2 = i
                
            if (first and p2 != -1 and p1 == p2-1 and word[p1][1] == "r" and word[p2][1] == "r"):
                word[p1] = [1,"b"]
                word[p2] = [2,"r"]
                pos = p1
                break
                
            elif (p2 != -1 and p1 == p2-1 and word[p1][1] == "r" and word[p2][1] == "r" 
                  and len([j for j in range(p2+1,len(word)) if word[j][0] == 2])==0):
                pos = p1
                word[p1] = [0,"x"]
                word[p2] = [2,"r"]
                break
                
            elif (p2 != -1 and p1 == p2-1 and word[p1][1] == "r" and word[p2][1] == "b" 
                  and len([j for j in range(p2+1,len(word)) if word[j][0] == 2])==0):
                pos = p1
                word[p1] = [0,"x"]
                word[p2] = [2,"r"]
                break
            
            else:
                if p2 != -1:
                    word[p2] = [0,"x"]
                    if p1 != -1:
                        word[p1] = [0,"x"]     
                    break
        if p2==-1:
            boo = False
    return([word,pos])
    
# Shifted collapse of two rows

# collapse row 1 on top of row 2 with shifted rules
# this method updates row1 and row2
# wit is a boolean variable that says if a collapse occurred or not
def shifted_two_row_collapse(row1,row2,first):
    par_word = []
    wit = False
    # Create the parenthesis word to do the matching algorithm
    for i in reversed(range(len(row1))):
        if row1[i]==2:
            par_word.append([1,"b"])
        elif row1[i]==-2:
            par_word.append([1,"r"])
        else:
            par_word.append([0,"x"])
        if row2[i]==-2:
            par_word.append([2,"r"])
        elif row2[i]==2:
            par_word.append([2,"b"])
        else:
            par_word.append([0,"x"])
        
    # Apply parenthesis matching
    [red_par_word,pos] = shifted_match_parenthesis(par_word,first)
    # Find indices on row1 to collapse
    coll = [red_par_word[i] for i in range(len(red_par_word)) if i%2==0] 

          
    l_coll_pos = len(row1)
    
    for i in range(len(row1)):
        if coll[i][0] == 1:
            wit = True
            l_coll_pos = i
            row1[len(row1)-1-i] = 0
            if coll[i][1] == "r":
                row2[len(row1)-1-i] = -2
            elif coll[i][1] == "b":
                row2[len(row1)-1-i] = 2
    
    doub = -1
    
    if pos != -1 and first:
        k = int(pos/2.0)
        row1[len(row1)-1-k] = 2
        l_coll_pos = k
        
    if pos != -1:
        doub = int(pos/2.0)
                
    return(row1.copy(),row2.copy(),wit,doub,l_coll_pos)


# Checks if a given arrangement of balls is valid to be a ShMLQ
# The array might have rows of 0s on top

def is_shifted_valid(balls):
    l = len(balls[0])
    h = len(balls)
    
    answer = True
    
    last_row_index = max([j for j in range(h) if balls[j] != [0 for z in range(l)]])
    
    # First check sizes of the balls
    if(len([j for j in range(l) if balls[last_row_index][j] != 0]) != 1):
        return(False)
    
    for i in range(h-1):
        if not ( (len(balls[i+1]) - len(balls[i])) in [0,1]):
            return(False)    
    
    # Second condition: every pair of balls has someone in the middle upstairs
    for r in range(h-1):
        row_pos = [j for j in range(l) if balls[r][j] != 0 ]
        
        for p in range(len(row_pos)-1):
            pos = row_pos[p]
            next_pos = row_pos[p+1]
            if(len( [k for k in range(pos,next_pos+1) if balls[r+1][k] != 0] )==0):
                return(False)
    
    return(answer)


# Hall-Littlewoood Polynomials 

def create_rarm_matrix(balls):
    mat = copy.deepcopy(balls)

    for i in range(len(balls)):
        for j in range(len(balls[0])):
            if i==0:
                mat[i][j] = 0
            else:
                if balls[i][j] == 1:
                    mat[i][j] = len([1 for k in range(j+1,len(balls[0])) if balls[i-1][k] == 1]) - len([1 for k in range(j+1,len(balls[0])) if balls[i][k] == 1])

    return(mat)

def create_rarm_prime_matrix(balls):
    mat = copy.deepcopy(balls)

    for i in range(len(balls)):
        for j in range(len(balls[0])):
            if i==0:
                mat[i][j] = 0
            else:
                if balls[i][j] == 1:
                    mat[i][j] = len([1 for k in range(j,len(balls[0])) if balls[i-1][k] == 1]) - len([1 for k in range(j,len(balls[0])) if balls[i][k] == 1])

    return(mat)

# Print a list of lists in a nice way
def print_list_of_lists(L):
    #print("--------------------------")
    print('\n'.join(' '.join('%2d' % x for x in l) for l in L))
    #print("--------------------------")

##   MULTILINE QUEUE CLASS   ##

class MultilineQueue:
    queues = []
    case = 0
    r = 0
    set_case = False
    
    # balls may be valid (partition content)
    # size = [width,height]
    def __init__(self,balls):
        self.balls = balls
        self.size = [len(balls[0]),len(balls)]
        self.balls_coordinates = balls_coordinates(balls)

        self.rarm_matrix = create_rarm_matrix(balls)
        self.rarm_prime_matrix = create_rarm_prime_matrix(balls)
                
        self.QM =[ [] for _ in range(self.size[1]) ]
        
    # This method creates the queues
    # 
    # case=0:   pairing to the right with (possibly) wrapping conditions
    #           pairing is done left to right in each row and priority order is respected
    #           queues are given as sequences of coordinates starting from the top
    #
    # case=1:   collapsing for The Game!
    #
    # case=2:   minimal distance insertion-pairing to the right with no-wrapping
    #           inserts each ball and updates the recording tableaux
    #           
    # case=3:   pairing to the left with (possibly) wrapping conditions
    #           pairing is done right to left in each row and priority order is respected
    #           queues are given as sequences of coordinates starting from the top
    #
    # case=4:   minimal distance insertion-pairing to the left with no-wrapping
    #           inserts each ball and updates the recording tableaux
    #
    # case=5:   collapsing with cyclic pairing conditions 
    #          
    #          

    
    def pair(self,case):
        if not self.set_case:
            self.case = case
            self.set_case = True
        self.queues = []
        if case==0:       
            
            to_queue = []
            w = self.size[1]
            l = self.size[0]
            for i in range(w):
                for j in reversed(range(l)):
                    if self.balls[i][j] == 1: 
                        to_queue.append([i+1,j+1])
                        
            cont = True
            while cont:
                b = to_queue[-1]
                queue = [b]
                to_queue.remove(b)
                done = False
                b0 = b.copy()
                while not done:
                    j = b0[0]-1
                    l = [m for m in range(b0[1],self.size[0]+1) if ([j,m] in to_queue)]
                    #if b0 is at the bottom
                    if b0[0] == 1:
                        done = True
                    #otherwise
                    else:
                        #case 1: wrapping
                        if l==[]:
                            c = min([m for m in range(0,b0[1]) if ([j,m] in to_queue)])
                        #case 2: non-wrapping
                        else:
                            c = min(l)
                        queue.append([j,c])
                        to_queue.remove([j,c])
                        b0 = [j,c]
                self.queues.append(queue)
                if len(to_queue) == 0:
                    cont = False  
        
        if case == 1:
            self.collapse_game()
            while (self.size[0])*[0] in self.balls:
                self.balls.remove((self.size[0])*[0])
            self.size = [len(self.balls[0]),len(self.balls)]
            self.balls_coordinates = balls_coordinates(self.balls)
            #self.pair(0)
            
        if case == 2:
            self.recording_and_collapse()
            self.size = [len(self.balls[0]),len(self.balls)]
            self.balls_coordinates = balls_coordinates(self.balls)
            self.pair(0)
            
        if case == 3:
            rev_balls = []
            for b in self.balls:
                rev_balls.append(list(reversed(b)))
            self.balls_coordinates = balls_coordinates(rev_balls)
            self.pair(0)
            
            self.balls_coordinates = balls_coordinates(self.balls)
            rev_queues = []
            for queue in self.queues:
                rev_q = []
                for p in queue:
                    rev_q.append([p[0],self.size[0]+1-p[1]])
                rev_queues.append(rev_q)
            
            self.queues = rev_queues
            
        if case == 4:
            rev_balls = []
            for b in self.balls:
                rev_balls.append(list(reversed(b)))
            self.balls_coordinates = balls_coordinates(rev_balls)
            self.pair(2)
            
            rb = []
            for b in self.balls:
                rb.append(list(reversed(b)))
            self.balls = rb
            self.balls_coordinates = balls_coordinates(self.balls)
            rev_queues = []
            for queue in self.queues:
                rev_q = []
                for p in queue:
                    rev_q.append([p[0],self.size[0]+1-p[1]])
                rev_queues.append(rev_q)
            
            self.queues = rev_queues
            
        if case == 5:
            self.recording_and_collapse_cyclic()
            self.size = [len(self.balls[0]),len(self.balls)]
            self.balls_coordinates = balls_coordinates(self.balls)
            self.pair(0)
                
                
    # Collapsing procedure for the ball arrangement.
    # WARNING: This changes the original balls so that the new arrangement is non-wrapping
    #          
    
    def collapse(self):
        L = self.size[1]
        for i in range(1,L):
            for j in reversed(range(1,i+1)):
                row1 = balls[j-1].copy()
                row2 = balls[j].copy()
                [balls[j],balls[j-1],wit] = two_row_collapse(row2,row1)
                
                
    # Collapsing procedure for The Game!
    #          
    
    def collapse_game(self):
        L = self.size[1]
        for i in range(1,L):
            for j in reversed(range(1,i+1)):
                row1 = self.balls[j-1].copy()
                row2 = self.balls[j].copy()
                [self.balls[j],self.balls[j-1],wit] = two_row_collapse_game(row2,row1)
    
                
    # Compute the recording tableau in the collapsing procedure (case 1) and collapse the ball arrangement.
    
    def recording_and_collapse(self):
        collapsed_balls = [(self.size[0])*[0]]
        for b in self.balls_coordinates:
            new_row = (self.size[0])*[0]
            new_row[b[1]-1] = 1
            
            collapsed_balls.append(new_row)
            
            for j in reversed(range(1,len(collapsed_balls))):
                row1 = collapsed_balls[j-1].copy()
                row2 = collapsed_balls[j].copy()
                [r2,r1,wit] = two_row_collapse(row2,row1)
                if not wit:
                    self.QM[j].append(b[0])
                    break
                elif j == 1:
                    self.QM[j-1].append(b[0])
                [collapsed_balls[j],collapsed_balls[j-1]] = [r2,r1]
                
            while (self.size[0])*[0] in collapsed_balls:
                collapsed_balls.remove((self.size[0])*[0])
        
        while [] in self.QM:
            self.QM.remove([])
            
        self.balls = collapsed_balls.copy()
        
        # Compute the recording tableau in the collapsing procedure (case 1) and collapse the ball arrangement.
    
    def recording_and_collapse_cyclic(self):
        L = self.size[1]
        prev_shape = [0 for l in range(L)]
        prev_shape[0] = sum(self.balls[0])
        
        for a in range(sum(self.balls[0])):
            self.QM[0].append(1)
            
        for i in range(1,L):
            for j in reversed(range(1,i+1)):
                row1 = self.balls[j-1].copy()
                row2 = self.balls[j].copy()
                [self.balls[j],self.balls[j-1],wit] = two_row_collapse_cyclic(row2,row1)
            
            cur_shape = [sum(b) for b in self.balls]
            
            for j in range(i+1):
                cant_j = cur_shape[j]-prev_shape[j]
                for k in range(cant_j):
                    self.QM[j].append(i+1)
                prev_shape[j] = cur_shape[j]
         
        while [] in self.QM:
            self.QM.remove([])       
        
        
    ##  SHIFTED METHODS  ##
    
    def shifted_collapse(self):
        L = self.size[1]
        for i in range(1,L):
            for j in reversed(range(1,i+1)):
                row1 = self.balls[j-1].copy()
                row2 = self.balls[j].copy()
                [balls[j],balls[j-1],wit] = shifted_two_row_collapse(row2,row1)
    
    
    # Shifted version of collapsing
    
    def shifted_recording_and_collapse(self):
        collapsed_balls = [(self.size[0])*[0]]
        for b in self.balls_coordinates:
            new_row = (self.size[0])*[0]
            if b[2] == "b":    
                new_row[b[1]-1] = 2
            elif b[2] == "r":    
                new_row[b[1]-1] = -2
            
            collapsed_balls.append(new_row)
            
            print("Initial balls")
            print_list_of_lists(list(reversed(collapsed_balls)))
            print("+++")
            
            pos_coll_fr = 0
            
            for j in reversed(range(1,len(collapsed_balls))):
                row1 = collapsed_balls[j-1].copy()
                row2 = collapsed_balls[j].copy()
                first = (j==1)
                [r2,r1,wit,pos,l_coll_pos] = shifted_two_row_collapse(row2,row1,first)
                
                if first:
                    pos_coll_fr = self.size[0]-1-l_coll_pos
                #if not wit:
                #    self.QM[j].append(b[0])
                #    break
                #elif j == 1:
                #    self.QM[j-1].append(b[0])
                
                if pos != -1:
                    r1_col_pos = collapsed_balls[j-1][self.size[0]-1-pos]
                    r2_col_pos = collapsed_balls[j][self.size[0]-1-pos]
                
                    print("pos")
                    print(pos)
                    print("row cols")
                    print(r1_col_pos)
                    print(r2_col_pos)
                            
                [collapsed_balls[j],collapsed_balls[j-1]] = [r2,r1]              
                
                if pos != -1 and (not first):
                    collapsed_balls[j][self.size[0]-1-pos] = 0
                    collapsed_balls[j-1][self.size[0]-1-pos] = r2_col_pos
                    collapsed_balls[j-2][self.size[0]-1-pos] = r1_col_pos
                
                if pos != -1 and r1_col_pos != r2_col_pos:
                    collapsed_balls[j][self.size[0]-1-pos] = 0
                    collapsed_balls[j-1][self.size[0]-1-pos] = r2_col_pos
                    collapsed_balls[j-2][self.size[0]-1-pos] = r1_col_pos
                    
            print("After collapsing")
            print_list_of_lists(list(reversed(collapsed_balls)))
            
            if pos_coll_fr != -1:
                possibs = [n for n in range(pos_coll_fr+1,self.size[0]) if (collapsed_balls[0][n] != 0)]
                
                if(len(possibs) > 0):
                    lift_pos = min(possibs)
                    num_to_lift = collapsed_balls[0][lift_pos]
                    for k in range(len(collapsed_balls)-1):
                        if collapsed_balls[k+1][lift_pos] == 0:
                            collapsed_balls[k+1][lift_pos] = num_to_lift
                            collapsed_balls[k][lift_pos] = 0   

                        else:
                            lift_pos = min([n for n in range(lift_pos+1,self.size[0]) if (collapsed_balls[k+1][n] != 0)])
                            num_to_lift = collapsed_balls[k][lift_pos].copy()
                            collapsed_balls[k+1][lift_pos] = num_to_lift
                            collapsed_balls[k][lift_pos] = 0

                        partial = [ball_row[:lift_pos+1] for ball_row in collapsed_balls]    
                        
                        #print("---")
                        #print_list_of_lists(list(reversed(partial)))
                        #print(is_shifted_valid(partial))
                        
                        if(is_shifted_valid(partial)):
                            possibs2 = [n for n in range(lift_pos+1,self.size[0]) if (collapsed_balls[k+1][n] != 0)]
                            if len(possibs2) > 0:
                                lift_pos = min(possibs2)
                        else:
                            lift_pos = min([n for n in range(lift_pos,self.size[0]) if (collapsed_balls[k+1][n] != 0)])
            
            
            print("After bounce")
            print_list_of_lists(list(reversed(collapsed_balls)))
            print("---")
            
            
            while (self.size[0])*[0] in collapsed_balls:
                collapsed_balls.remove((self.size[0])*[0])
            
            
        while [] in self.QM:
            self.QM.remove([])
            
        self.balls = collapsed_balls.copy()
        
    
    # Draw a queue between pi and pf in a drawing win
    # queues must be ordered from top to bottom
    # case : see pairing cases

    def draw_queue(self,win,pi,pf,case,offset):
        r = 0.2
        xi = pi[1]-0.5
        yi = pi[0]-0.5
        xf = pf[1]-0.5
        yf = pf[0]-0.5
        if case == 0 or case == 1 or case == 2 or case == 5:
            if xi <= xf:
                win += line([(xi,yi-self.r),(xi,yi-0.5+offset)],color=Color('black'))
                win += line([(xi,yi-0.5+offset),(xf,yi-0.5+offset)],color=Color('black'))
                win += line([(xf,yi-0.5+offset),(xf,yf+self.r)],color=Color('black'))
            else:
                win += line([(xi,yi-self.r),(xi,yi-0.5+offset)],color=Color('black'))
                win += line([(xi,yi-0.5+offset),(self.size[0],yi-0.5+offset)],color=Color('black'))
                win += line([(0,yi-0.5+offset),(xf,yi-0.5+offset)],color=Color('black'))
                win += line([(xf,yi-0.5+offset),(xf,yf+self.r)],color=Color('black'))
            return(win)
        
        elif case == 3 or case == 4 or case == 6 or case == 7:
            if xi < xf:                
                win += line([(xi,yi-self.r),(xi,yi-0.5+offset)],color=Color('black'))
                win += line([(xi,yi-0.5+offset),(0,yi-0.5+offset)],color=Color('black'))
                win += line([(self.size[0],yi-0.5+offset),(xf,yi-0.5+offset)],color=Color('black'))
                win += line([(xf,yi-0.5+offset),(xf,yf+self.r)],color=Color('black'))

            else:               
                win += line([(xi,yi-self.r),(xi,yi-0.5+offset)],color=Color('black'))
                win += line([(xi,yi-0.5+offset),(xf,yi-0.5+offset)],color=Color('black'))
                win += line([(xf,yi-0.5+offset),(xf,yf+self.r)],color=Color('black'))
            return(win)
            
                    
    # Gives the drawing of a MLQ that can be paired or not (in which case just shows the ball arrangement)             
    def draw(self,radius):
        win = Graphics()
        self.r = radius
        w = self.size[1]
        h = self.size[0]
        
        #Draw gray grid        
        for i in range(0,h+1,1):
            win += line([(i,0),(i,w)],color=Color('lightgray'))
        for i in range(0,w+1,1):    
            win += line([(0,i),(h,i)],color=Color('lightgray'))
        
        #Draw ball arrangement
        for b in self.balls_coordinates:
            yb = b[0]
            xb = b[1]
            stop = False
            lab = str("")
            for queue in self.queues:
                if(b in queue):
                    lab = str(len(queue))
                if(stop):
                    break
                
            if len(b) == 2:
                win+= circle((xb-0.5,yb-0.5), self.r, fill = True, color = Color('black'))
                
                #win+= circle((xb-0.5,yb-0.5), self.r, fill = False, color = Color('black'))
                #win+= text(lab,(xb-0.5,yb-0.525),color = Color('black'),fontsize = 18,rotation = 0)
            elif len(b) == 3:
                if b[2] == "r":
                    win+= circle((xb-0.5,yb-0.5), self.r, fill = True, color = Color('red'))
                elif b[2] == "b":
                    win+= circle((xb-0.5,yb-0.5), self.r, fill = True, color = Color('black'))
                                
        #Draw queues
        for k in range(len(self.queues)):
            queue = self.queues[k] 
            for j in range(1,len(queue)):
                N = len(self.queues)
                #maximum offset to avoid queues to be intersecting in the drawing
                par = (-1)/(5.0)
                #offset function
                off = (par)-(2*par/(1.0*N))*(k)
                #draw the line between elements of the queue
                win = self.draw_queue(win,queue[j-1],queue[j],self.case,off)

        win.axes(False)
        win.show()
        
        
    def draw_board_game(self,radius):
        win = Graphics()
        self.r = radius
        w = self.size[1]
        h = self.size[0]
        
        #Draw gray grid        
        for i in range(0,h+1,1):
            win += line([(i,0),(i,w)],color=Color('lightgray'))
        for i in range(0,w+1,1):    
            win += line([(0,i),(h,i)],color=Color('lightgray'))
        
        #Draw column numbers
        for i in range(self.size[0]):
            win+= text(str(i+1),(i+0.5,-0.3),color = Color('gray'),fontsize = 12,rotation = 0)
        
        
        #Draw ball arrangement
        for i in range(len(self.balls)):
            current_row = self.balls[i]
            for j in range(len(current_row)):
                elem = current_row[j]
                colour = elem[1]
                if(elem[0]==1):
                    yb = i+1
                    xb = j+1
                    win+= circle((xb-0.5,yb-0.5), self.r, fill = True, color = Color(colour))

        win.axes(False)
        win.show()
        
    def partition(self):
        lam_prime = [sum(b) for b in self.balls]
        lam_p = Partition(lam_prime)
        return(lam_p.conjugate())
        
        
    ##  ALGEBRAIC METHODS  ##
    
    #returns the x-weight of the multiline queue. Does not need to be paired
    def xweight(self):
        wei = xs[0]**0
        for b in self.balls_coordinates:
            wei *= xs[b[1]-1]
        return(wei)
    
    #returns the x-weight of the multiline queue. Requires pairing
    # case: see pairing cases
    def qweight(self):
        wei = q**0
        #q statistic calculation depends on the pairing algorithm
        if self.case == 0 or self.case == 1 or self.case==5:
            for queue in self.queues:
                l = queue[0][0]
                for i in range(1,len(queue)):
                    ci = queue[i-1][1]
                    cf = queue[i][1]
                    if cf < ci:
                        wei *= q**(l+1-queue[i-1][0]) 
                        
        elif self.case == 3 or self.case == 4:
            for queue in self.queues:
                l = queue[0][0]
                for i in range(1,len(queue)):
                    ci = queue[i-1][1]
                    cf = queue[i][1]
                    if cf > ci:
                        wei *= q**(l+1-queue[i-1][0])
        return(wei)
    
    #returns the x-weight of the multiline queue. Requires pairing
    # case: see pairing cases
    def q_coweight(self):
        wei = q**0
        #q statistic calculation depends on the pairing algorithm
        if self.case == 0 or self.case == 1 or self.case == 5:
            for queue in self.queues:
                cont = 0
                for i in reversed(range(len(queue)-1)):
                    ci = queue[i][1]
                    cf = queue[i+1][1]
                    if cf >= ci:
                        cont += 1
                
                    wei *= q**(cont)
                        
                        
        elif self.case == 3 or self.case == 4:
            for queue in self.queues:
                cont = 0
                for i in reversed(range(len(queue)-1)):
                    ci = queue[i][1]
                    cf = queue[i+1][1]
                    if cf <= ci:
                        cont += 1
                    wei *= q**(cont)
        return(wei)
    
    #returns the column word of the multiline queue: column number of the balls from left to right, and bottom to top
    def rw(self):
        w = []
        for i in range(self.size[1]):
            for j in (range(self.size[0])):
                if self.balls[i][j] == 1:
                    w.append(j+1)
        
        return(w)
    
    #returns the column word of the multiline queue: row number of the balls from top to bottom and left to right
    def cw(self):
        w = []
        for i in range(self.size[0]):
            for j in reversed(range(self.size[1])):
                if self.balls[j][i] == 1:
                    w.append(j+1)
        
        return(w)
    
    #returns the inverted column word of the multiline queue: row number of the balls from top to bottom and right to left
    def invcolw(self):
        w = []
        for i in reversed(range(self.size[0])):
            for j in reversed(range(self.size[1])):
                if self.balls[j][i] == 1:
                    w.append(j+1)
                return(w)

    def tweight(self):
        w = 1
        for i in range(self.size[1]):
            for j in range(self.size[0]):
                if self.rarm_matrix[i][j] > 0:
                    if self.balls[i-1][j] == 0:
                        w *= (1-t^(self.rarm_matrix[i][j]))
        
        return(w)

In [3]:
# Matrix rotations

# rotate the ball arrangement 90 degrees CLOCKWISE
def rotate_clockwise(balls):
    balls2 = [[0 for i in range(len(balls))] for j in range((len(balls[0])))]
    for i in range(len(balls)):
        for j in range(len(balls[0])):
            if balls[i][j] == 1:
                balls2[j][i] = 1
        
    return(list(reversed(balls2)))

# rotate the ball arrangement 90 degrees COUNTER-CLOCKWISE
def rotate_counterclockwise(balls):
    w = len(balls[0])
    h = len(balls)
    balls2 = [[0 for i in range(h)] for j in range(w)]
    for i in range(h):
        for j in range(w):
            if balls[i][j] == 1:
                balls2[j][h-i-1] = 1
        
    return(balls2)
    

# balls is a ball arrangement
# return the corresponding pair of multiline queues for the given case of pairing
def mRSK(balls, case): 
    r = 0.2
    balls1 = balls.copy()
    balls2 = rotate_counterclockwise(balls.copy())
    
    M1 = MultilineQueue(balls1)
    print("Original binary matrix")
    M1.draw(r)
    
    print("Collapse Down")
    M1.pair(case)
    M1.draw(r)
    print("Recording Tableau")
    print_list_of_lists(reversed(M1.QM))
    print("\n")
    
    M2 = MultilineQueue(balls2)
    print("Collapse Left")
    M2.pair(case)
    M2.draw(r)
    print("Recording Tableau")
    print_list_of_lists(reversed(M2.QM))
    print("\n")
    
    # Double collapsing #1
    balls3 = rotate_clockwise(M2.balls.copy())
    M3 = MultilineQueue(balls3)
    print("Collapse Left-Down")
    M3.pair(case)
    M3.draw(r)
    print("Recording Tableau")
    print_list_of_lists(reversed(M3.QM))
    print("\n")
    
    # Double collapsing #2
    balls4 = rotate_counterclockwise(M1.balls.copy())
    M4 = MultilineQueue(balls4)
    print("Collapse Down-Right")
    M4.pair(case)
    M4.draw(r)
    print("Recording Tableau")
    print_list_of_lists(reversed(M4.QM))
    print("\n")
    
    #print("Check that the inverse works:")
    #balls5 = mRSK_inverse(M1,list(M3.QM),2)
    #M5 = MultilineQueue(balls)
    #M5.draw(r)
    
    
# Auxiliary methods for inverse map

# word is a ternary word with 0,1,2 and 1=(, 2=) and 0=blank, to perform parenthesis matching algorithm
# returns the unmatched letters from row2 after matching all possible 1s and 2s
def match_parenthesis_lift(w):
    word = w.copy()
    boo = True
    while boo:
        p1 = -1
        p2 = -1
        for i in range(len(word)):
            if word[i] == 1:
                p1 = i
            elif word[i] == 2:
                p2 = i
            
            if p2 != -1 and p1 != -1 and p1<p2:
                word[p2] = 0
                word[p1] = 0 
                break
        if len([i for i in range(len(word)) if word[i]==1])==0:
            boo = False
    return(word)


# Lift rightmost ball that is unpaired from row2 to row1
def two_row_lift(row1,row2):
    par_word = []
    # Create the "inverse" parenthesis word to do the matching algorithm
    for i in reversed(range(len(row1))):
        if row2[i]==1:
            par_word.append(1)
        else:
            par_word.append(0)
        if row1[i]==1:
            par_word.append(2)
        else:
            par_word.append(0)
    # Apply parenthesis matching
    red_par_word = match_parenthesis(par_word)
    w1 = [red_par_word[i] for i in range(len(red_par_word)) if i%2==0] 
    w1 = list(reversed(w1))
    # Find index on row2 to lift
    indices = [j for j in range(len(w1)) if w1[j] == 1]
    if indices != []:
        pos = max(indices)
        row2[pos] = 0
        row1[pos] = 1
    
    

# Compute the binary matrix associated to a multiline queue M and a (recording) tableau Qtab
# M and Qtab need have matching sizes
def mRSK_inverse(M,Qtab,case):
    if case == 2:
        balls = M.balls.copy()
        w = len(balls[0])
        h = len(balls)
        rec = Qtab.copy()
        while rec != []:
            M = max([max(rec[i]) for i in range(len(rec))])
            row = min([j for j in range(len(rec)) if M in rec[j]])
            aux = list(reversed(rec[row]))
            aux.remove(M)
            rec[row] = list(reversed(aux))
            for k in range(M-len(balls)):
                balls.append([0 for i in range(w)])
            for i in range(row,M-1):
                row1 = balls[i+1]
                row2 = balls[i]
                two_row_lift(row1,row2)
            while [] in rec:
                rec.remove([])
        
        return(balls)   
    

def mlq(tableau,case):
    rw = list(reversed(tableau.to_word_by_column()))
    balls = []
    M = max(rw)
    for i in rw:
        row = [0 for j in range(M)]
        row[i-1] = 1
        balls.append(row)
    print("\n")
    print("Tableau:")
    M = MultilineQueue(balls)
    Tableaux.options.convention="french"
    tableau.pp()
    print("\n")
    print("Multiline queue:")
    print("\n")
    M.pair(case)
    M.draw(0.2)
    #print("Recording tableau:")
    #print_list_of_lists(reversed(M.QM))
    return(M)

def tab(M):
    w = M.rw()
    t0 = Tableau([])
    w = list(reversed(w))
    t = t0.insert_word(w,left=True)
    t.pp()
    return(t)

## Algebraic combinatorics

In [4]:
##   Schur functions   ##

def schur_mlq(n,shape):
    R=PolynomialRing(coeffs_field,n,'x')
    xs = R.gens()

    part = Partition(shape)
    mlq_shape = list(part.conjugate())

    aux_list = []
    ball_arrangements = []

    for i in mlq_shape:
        aux_list.append([list(sub) for sub in Subsets(range(n)) if len(sub) == i])

    ball_arr_coords = itertools.product(*aux_list)

    for tup in ball_arr_coords:
        ball_arr = []
        for row in tup:
            ball_row = [0 for j in range(n)]
            for i in range(n):
                if i in row:
                    ball_row[i] = 1
            ball_arr.append(ball_row.copy())
        ball_arrangements.append(ball_arr)


    # Create all MLQs with n columns of given shape & compute the polynomial
    poly = 0

    for balls in ball_arrangements:
        M = MultilineQueue(balls.copy())
        M.pair(0)
        if M.qweight() == 1:
            #r=0.2
            #M.draw(r)
            poly += M.xweight()

    # Convert polynomial to symmetric function
    f = sym.from_polynomial(poly)
    return(s(f))


##  q--Whittaker polynomials  ##

def charge_gen(n,lam):
    R=PolynomialRing(coeffs_field,n,'x')
    xs = R.gens()

    part = Partition(lam)
    mlq_shape = list(part.conjugate())

    aux_list = []
    ball_arrangements = []

    for i in mlq_shape:
        aux_list.append([list(sub) for sub in Subsets(range(n)) if len(sub) == i])

    ball_arr_coords = itertools.product(*aux_list)

    for tup in ball_arr_coords:
        ball_arr = []
        for row in tup:
            ball_row = [0 for j in range(n)]
            for i in range(n):
                if i in row:
                    ball_row[i] = 1
            ball_arr.append(ball_row.copy())
        ball_arrangements.append(ball_arr)


    # Create all MLQs with n columns of given shape & compute the polynomial
    poly = 0

    for balls in ball_arrangements:
        M = MultilineQueue(balls.copy())
        M.pair(0)
        xM = M.xweight()
        poly += M.qweight()*xM

    # Convert polynomial to symmetric function
    return(poly)


##  ???  ##

def cocharge_gen(n,lam):
    R=PolynomialRing(coeffs_field,n,'x')
    xs = R.gens()

    part = Partition(lam)
    mlq_shape = list(part.conjugate())

    aux_list = []
    ball_arrangements = []

    for i in mlq_shape:
        aux_list.append([list(sub) for sub in Subsets(range(n)) if len(sub) == i])

    ball_arr_coords = itertools.product(*aux_list)

    for tup in ball_arr_coords:
        ball_arr = []
        for row in tup:
            ball_row = [0 for j in range(n)]
            for i in range(n):
                if i in row:
                    ball_row[i] = 1
            ball_arr.append(ball_row.copy())
        ball_arrangements.append(ball_arr)


    # Create all MLQs with n columns of given shape & compute the polynomial
    poly = 0

    for balls in ball_arrangements:
        M = MultilineQueue(balls.copy())
        M.pair(0)
        xM = M.xweight()
        poly += M.q_coweight()*xM

    # Convert polynomial to symmetric function
    return(poly)

# Other methods #

def exponent(l):
    ans = xs[0]**0
    for i in range(len(l)):
        m = int(l[i])
        ans = ans*xs[i]**m
    return(ans)


##   Kostka-Faulkes polynomials   ##

def KostKaFoulkesPol_mlq(n,lam,mu):
    R=PolynomialRing(coeffs_field,n,'x')
    xs = R.gens()

    part = Partition(lam)
    mlq_shape = list(part.conjugate())

    aux_list = []
    ball_arrangements = []

    for i in mlq_shape:
        aux_list.append([list(sub) for sub in Subsets(range(n)) if len(sub) == i])

    ball_arr_coords = itertools.product(*aux_list)

    for tup in ball_arr_coords:
        ball_arr = []
        for row in tup:
            ball_row = [0 for j in range(n)]
            for i in range(n):
                if i in row:
                    ball_row[i] = 1
            ball_arr.append(ball_row.copy())
        ball_arrangements.append(ball_arr)


    # Create all MLQs with n columns of given shape & compute the polynomial
    poly = 0

    for balls in ball_arrangements:
        M = MultilineQueue(balls.copy())
        M.pair(0)
        
        xM = M.xweight()
        
        if xM == exponent(mu) and M.qweight() == 1:
            r=0.2
            M.draw(r)
            b = M.balls.copy()
            bN = rotate_counterclockwise(b)
            N = MultilineQueue(bN)
            N.pair(0)
            poly += N.qweight()

    # Convert polynomial to symmetric function
    return(poly)


def KostKaFoulkesPol_mlq2(n,lam,mu):
    R=PolynomialRing(coeffs_field,n,'x')
    xs = R.gens()

    part = Partition(mu)
    mlq_shape = list(part)

    aux_list = []
    ball_arrangements = []

    for i in mlq_shape:
        aux_list.append([list(sub) for sub in Subsets(range(n)) if len(sub) == i])

    ball_arr_coords = itertools.product(*aux_list)

    for tup in ball_arr_coords:
        ball_arr = []
        for row in tup:
            ball_row = [0 for j in range(n)]
            for i in range(n):
                if i in row:
                    ball_row[i] = 1
            ball_arr.append(ball_row.copy())
        ball_arrangements.append(ball_arr)


    # Create all MLQs with n columns of given shape & compute the polynomial
    poly = 0

    for balls in ball_arrangements:
        M = MultilineQueue(balls.copy())
        M.pair(0)
        
        xM = M.xweight()
        
        if xM == exponent(list(lam.conjugate())):
            b = M.balls.copy()
            N = MultilineQueue(b)
            N.pair(2)
            if(N.partition() == lam.conjugate()):
                M.draw(0.2)
                #print(M.rw())
                #N.draw(0.2)
                poly += M.qweight()

    # Convert polynomial to symmetric function
    return(poly)

def modified_KostKaFoulkesPol_mlq2(n,lam,mu):
    R=PolynomialRing(coeffs_field,n,'x')
    xs = R.gens()

    part = Partition(mu)
    mlq_shape = list(part.conjugate())

    aux_list = []
    ball_arrangements = []

    for i in mlq_shape:
        aux_list.append([list(sub) for sub in Subsets(range(n)) if len(sub) == i])

    ball_arr_coords = itertools.product(*aux_list)

    for tup in ball_arr_coords:
        ball_arr = []
        for row in tup:
            ball_row = [0 for j in range(n)]
            for i in range(n):
                if i in row:
                    ball_row[i] = 1
            ball_arr.append(ball_row.copy())
        ball_arrangements.append(ball_arr)


    # Create all MLQs with n columns of given shape & compute the polynomial
    poly = 0

    for balls in ball_arrangements:
        M = MultilineQueue(balls.copy())
        M.pair(0)
        
        xM = M.xweight()
        
        if xM == exponent(list(lam.conjugate())):
            b = M.balls.copy()
            N = MultilineQueue(b)
            N.pair(2)
            if(N.partition() == lam.conjugate()):
                #M.draw(0.2)
                #print(M.rw())
                #N.draw(0.2)
                poly += M.q_coweight()

    # Convert polynomial to symmetric function
    return(poly)

##  HALL--LITTLEWOOD POLYNOMIALS  ##

def is_all_positive(mat):
    ans = True
    for i in range(len(mat)):
        for j in range(len(mat[0])):
            if mat[i][j] < 0:
                ans= False
                break
        if not ans:
            break
    return(ans)


def all_non_wrapping_MLQs(n,lam):
    part = Partition(lam)
    mlq_shape = list(part.conjugate())

    aux_list = []
    ball_arrangements = []

    for i in mlq_shape:
        aux_list.append([list(sub) for sub in Subsets(range(n)) if len(sub) == i])

    ball_arr_coords = itertools.product(*aux_list)

    for tup in ball_arr_coords:
        ball_arr = []
        for row in tup:
            ball_row = [0 for j in range(n)]
            for i in range(n):
                if i in row:
                    ball_row[i] = 1
            ball_arr.append(ball_row.copy())
        ball_arrangements.append(ball_arr)


    # Create all MLQs with n columns of given shape

    for balls in ball_arrangements:
        M = MultilineQueue(balls.copy())
        M.pair(0)
        if is_all_positive(M.rarm_prime_matrix):
            r=0.2
            M.draw(r)

def all_non_wrapping_MLQs_partition_content(n,lam,mu):
    part = Partition(lam)
    mlq_shape = list(part.conjugate())

    aux_list = []
    ball_arrangements = []

    for i in mlq_shape:
        aux_list.append([list(sub) for sub in Subsets(range(n)) if len(sub) == i])

    ball_arr_coords = itertools.product(*aux_list)

    for tup in ball_arr_coords:
        ball_arr = []
        for row in tup:
            ball_row = [0 for j in range(n)]
            for i in range(n):
                if i in row:
                    ball_row[i] = 1
            ball_arr.append(ball_row.copy())
        ball_arrangements.append(ball_arr)


    # Create all MLQs with n columns of given shape

    for balls in ball_arrangements:
        M = MultilineQueue(balls.copy())
        M.pair(0)
        x_exp = list(M.xweight().exponents()[0])[0:n]
        if x_exp == mu+[0 for i in range(n-len(mu))]:
            if is_all_positive(M.rarm_prime_matrix):
                r=0.2
                M.draw(r)


def Hall_Littlewood(n,lam):
    R=PolynomialRing(coeffs_field,n,'x')
    xs = R.gens()
    
    part = Partition(lam)
    mlq_shape = list(part.conjugate())

    aux_list = []
    ball_arrangements = []

    for i in mlq_shape:
        aux_list.append([list(sub) for sub in Subsets(range(n)) if len(sub) == i])

    ball_arr_coords = itertools.product(*aux_list)

    for tup in ball_arr_coords:
        ball_arr = []
        for row in tup:
            ball_row = [0 for j in range(n)]
            for i in range(n):
                if i in row:
                    ball_row[i] = 1
            ball_arr.append(ball_row.copy())
        ball_arrangements.append(ball_arr)


    # Create all MLQs with n columns of given shape & compute the polynomial
    poly = 0

    for balls in ball_arrangements:
        M = MultilineQueue(balls.copy())
        M.pair(0)
        if is_all_positive(M.rarm_prime_matrix):
            # r=0.2
            # M.draw(r)
            # print(M.xweight()*M.tweight())
            poly += M.xweight()*M.tweight()

    return(poly)


In [11]:
# Lascoux-Schutzenberger Involution

def p_operator(T,i):
    S = copy.deepcopy(T)
    for j in reversed(range(1,i+1)):
        S = S.bender_knuth_involution(j)
    return(S)
        

def S_operator(T,i):
    S = copy.deepcopy(T)
    for j in range(1,i):
        S = p_operator(S,j)
    return(S)

def Lascoux_Schutzenberger_involution(T,i):
    return( S_operator(S_operator(T,i+1).bender_knuth_involution(1),i+1) )